In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_variable_cost_runs(acq, tfn, dim):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOpt", filters={
        "sweep": "zz8l7my8", 
        "config.lengthscale": 0.0001, 
        "config.kernel": "Matern52", 
        "config.dim": dim,
        "config.problem": tfn,
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = {
      'ExpectedImprovementWithoutCost':'ExpectedImprovementWithoutCost',
      'ExpectedImprovementPerUnitCost':'ExpectedImprovementPerUnitCost',
      'Gittins_Lambda_01':'Gittins_Lambda01',
      'Gittins_Lambda_001':'Gittins_Lambda001',
      'Gittins_Lambda_0001':'Gittins_Lambda0001'
    }
target_functions = ['hard_for_eipc', 'hard_for_ei']
dimensions = [1]

In [3]:
grouped_runs = {(a,t,d): load_bayesian_regret_variable_cost_runs(a,t,d) for a in acquisition_functions.keys() for t in target_functions for d in (dimensions)}

100%|██████████| 16/16 [00:12<00:00,  1.23it/s]


In [6]:
for a in acquisition_functions.keys():
    for t in target_functions:
        for d in dimensions:
            config_and_metrics_per_seed = grouped_runs[a,t,d]

            cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
            best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T

            # best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
            # best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
            # best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)

            # output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)

            global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed])

            regret_per_seed = global_optimum_per_seed - best_observed_per_seed

            print(a, t, d, regret_per_seed.shape)

            regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
            regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
            regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

            output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)

            np.savetxt(f"results/variable_amplitude/VariableAmplitude_{t}_d{d}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

ExpectedImprovementWithoutCost hard_for_eipc 1 (401, 16)
ExpectedImprovementWithoutCost hard_for_ei 1 (401, 16)
ExpectedImprovementPerUnitCost hard_for_eipc 1 (401, 16)
ExpectedImprovementPerUnitCost hard_for_ei 1 (401, 16)
Gittins_Lambda_01 hard_for_eipc 1 (401, 16)
Gittins_Lambda_01 hard_for_ei 1 (401, 16)
Gittins_Lambda_001 hard_for_eipc 1 (401, 16)
Gittins_Lambda_001 hard_for_ei 1 (401, 16)
Gittins_Lambda_0001 hard_for_eipc 1 (401, 16)
Gittins_Lambda_0001 hard_for_ei 1 (401, 16)
